In [77]:
import os
import time
from transformers import AutoFeatureExtractor, AutoModelForImageClassification, ViTImageProcessor
import torch
from PIL import Image
from sklearn.metrics import accuracy_score
import torch.onnx
import onnxruntime
import numpy

In [78]:
# Путь к нашему ViT, который будем конвертировать.
path_to_model = "weights/my_model"

extractor = ViTImageProcessor.from_pretrained(path_to_model)
vit_model = AutoModelForImageClassification.from_pretrained(path_to_model)

In [79]:
def model_use(model, img):
    with torch.no_grad():
        logits = model(**img).logits

    predicted_label = logits.argmax(-1).item()

    return model.config.id2label[predicted_label]


# Функция для запуска конвертированной в ONNX модели. 
def onnx_model_use(ort_session, img):
    
    model_id2label=  {0: "cats", 1: "dogs"}
    ort_session = ort_session
    ort_inputs = {ort_session.get_inputs()[0].name: img['pixel_values'].numpy()}
    ort_outs = ort_session.run(None, ort_inputs)[0]

    predicted_label = ort_outs.argmax(-1).item()

    return model_id2label[predicted_label]

In [80]:
# Путь к тестовым картинкам.
path_to_images = "data/"

images_list = os.listdir(path_to_images)

In [81]:
# Функция для замера размера модели.
def size_measurement(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()

    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()

    size_all_mb = (param_size + buffer_size) / (1024 ** 2)
    print('model size: {:.3f}MB'.format(size_all_mb))

In [82]:
# Найдем исходный размер модели.
size_measurement(vit_model)

model size: 327.302MB


In [83]:
path_to_onnx_model = "vit.onnx"
image = Image.open(path_to_images + images_list[0], mode='r', formats=None)
inputs_onnx = extractor(image, return_tensors="pt")

torch_out = vit_model(**inputs_onnx)

# Экспортируем модель в ONNX.
torch.onnx.export(vit_model,  # model being run
                  {'pixel_values': inputs_onnx['pixel_values']},  # model input (or a tuple for multiple inputs)
                  path_to_onnx_model,   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=11,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['pixel_values'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'pixel_values' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [84]:
# Проверим размер ONNX модели.
onnx_file_size = os.path.getsize(path_to_onnx_model)/(1024**2)

print("Размер модели после конвертации в ONNX стал ", onnx_file_size, " мегабайт.")

Размер модели после конвертации в ONNX стал  327.5526752471924  мегабайт.


In [85]:
# Загрузим ONNX модель.
ort_session = onnxruntime.InferenceSession(path_to_onnx_model)

In [86]:
# Запустим тест нашей ONNX модели.

start_time = time.time()

# Собака 1, кошка 0.
target_list = []
predict_list = []

for element in images_list:

    image = Image.open(path + element, mode='r', formats=None)

    inputs = extractor(image, return_tensors="pt")
    predict = onnx_model_use(ort_session, inputs)
    target = element[:element.find(".")]

    if target == "dog":
        label = 1
    else:
        label = 0

    target_list.append(label)

    if predict == "dogs":
        pr = 1
    else:
        pr = 0

    predict_list.append(pr)

end_time = time.time()

acc = accuracy_score(target_list, predict_list)
print("Точность сконвертированной в ONNX модели= ", acc)
print("Время обработки изображений сконвертированной в ONNX моделью = ", end_time-start_time, " секунд")
print("Скорость обработки изображений у сконвертированной в ONNX модели составила  ", len(images_list)/(end_time-start_time), " картинок в секунду")

Точность сконвертированной в ONNX модели=  0.9875
Время обработки изображений сконвертированной в ONNX моделью =  19.00001573562622  секунд
Скорость обработки изображений у сконвертированной в ONNX модели составила   8.421045657345955  картинок в секунду
